In [ ]:
import importlib

In [ ]:
import os
import json
from datetime import date, datetime, timedelta, time
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import pandas as pd
from plotly_tools import genIMGfromFile
from IPython.display import Image as ImageD
from sqlalchemy import create_engine, desc, asc
from sqlalchemy.orm import scoped_session, sessionmaker
import time

In [ ]:

import updateDB
importlib.reload(updateDB)
import libsstock
importlib.reload(libsstock)
import libs.datamodel
importlib.reload(libs.datamodel)
from updateDB import (
    updateOHLC, getStockIntradayData, getYFdata, updateDB, updateDBintradayFromSSI, getLastsValue, getStockName,
    getStockData, addStock, updateDB, getStockIntradayData, getStockName
)

from libsstock import (
    loadStocks, computeIchimoku,
    graphIchimoku, graphBestGain, graphWorseGain, graphCashLock, graphRendement,
    graphEvolutionTitre, checkVar,
    loadStocks, computeIchimoku,
    graphIchimoku, graphBestGain, graphWorseGain, graphCashLock, graphRendement,
    graphEvolutionTitre, checkVar, graphDataForStock, loadStocks, genLink1stock, detectStockVar,
    detectStockVar, computeRRvals, computeParms, timeFromOpenningMin, genLink1stock, isMarketOpen
)
from libs.datamodel import StockDay, StockIntraDay, HistDetectStockVar
#updateDB()
updateDBintradayFromSSI()
#updateDB(daysHisto=365*10, stockRes='daily')
#updateDB(daysHisto=6, stockRes='intraday')

In [ ]:
from yahoo_fin import stock_info as si

si.get_live_price('AI.PA')

In [ ]:
df = loadStocks('mystocks.json')
dfS = df[df['sellValue'].isna()].sort_values('boughtDate')
strOut = ''
for lineIND in dfS.index:
    line = dfS.loc[lineIND]
    strOut += line['name'] + " / " + line['stockname'] + " / " + "/stockinfo" + str(line['stockname']).replace('.', '_') + '\n'

    strOut += "- PRU      : {:.2f} €\n".format(line['boughtNetValue'])
    strOut += "- COURS    : {:.2f} €\n".format(line['valueNow'])
    strOut += "- CAPITAL  : {:.2f} €\n".format(line['netActualLock'])
    strOut += "- LOSS/GAIN: {:.2f} % / {:.2f} €\n".format(line['netActualGainPercent'], line['netActualGain'])
print(strOut)

In [ ]:
dfS = loadStocks('mystocks.json')
dfS = dfS[dfS['sellValue'].isna()].sort_values('boughtDate')

In [ ]:
dfS

In [ ]:

investValue = float('68')
boughtValue = float('9.16')
(boughtQ, investCash, valBuy) = computeParms(boughtValue, investValue)
rrVals = computeRRvals(valBuy, boughtValue, maxLoss=0.05, riskRewardRatio=2)
print('Ordre à seuil de déclenchement')
print('Q={:.0f}, PRU={:.3f} €, capital={:.3f} €, Frais: {:.1f}'.format(boughtQ, valBuy, investCash, rrVals['impactFrais'] * 100))
print('Stoploss: {:.2f} € / {:.1f} %'.format(rrVals['stopLossVal'], rrVals['stopLossRatio'] * 100))
print('Sell    : {:.2f} € / {:.1f} %'.format(rrVals['takeProfitVal'], rrVals['takeProfitRatio'] * 100))

In [ ]:
def computeRRgraph(stockName='EMG.PA', riskRewardRatio=2, maxLoss=0.05):
    boughtDate = dfS[dfS['stockname'] == stockName]['boughtDate']
    boughtValue = float(dfS[dfS['stockname'] == stockName]['boughtValue'])
    valBuy = float(dfS[dfS['stockname'] == stockName]['boughtNetValue'])
    qBuy = float(dfS[dfS['stockname'] == stockName]['boughtQ'])
    sumFrais = dfS[dfS['stockname'] == stockName]['boughtFrais'] + dfS[dfS['stockname'] == stockName]['sellFrais']
    stopLossPercent = maxLoss - (boughtValue - valBuy)/valBuy
    stopLossVal = boughtValue * (1 - stopLossPercent)
    sellRatioPercent = maxLoss * riskRewardRatio
    sellRatioVal = boughtValue * (1 + sellRatioPercent)
    df = getStockIntradayData(stockName, datetime.now() - timedelta(hours=15))
    df = updateOHLC(df, freq=15, unit='T')
    tmpstore = np.array(df['Close'])
    df['relGain'] = (tmpstore - valBuy) / valBuy * 100
    df['absGain'] = (tmpstore - valBuy) * qBuy
    df.index += timedelta(hours=1)
    dataFig = []
    dataFig.append({
            'x': df.index,
            'y': df['relGain'],
            'type': 'scatter',
    })
    dataFig.append({
            'x': [df.index[0], df.index[-1]],
            'y': [sellRatioPercent * 100, sellRatioPercent * 100],
            'type': 'scatter',
            'name': 'Sell barrier'
    })
    dataFig.append({
            'x': [df.index[0], df.index[-1]],
            'y': [- stopLossPercent * 100, - stopLossPercent * 100],
            'type': 'scatter',
            'name': 'Stop Loss'
    })
    dataFig.append({
        'x': [boughtDate.iloc[0] + timedelta(hours=1)],
        'y': [(boughtValue - valBuy)/valBuy*100],
        'type': 'scatter',
        'name': 'Sell', 'type': 'scatter', 'mode': 'markers',
        'marker': dict(
            color='orange',
            size=10,
            line=dict(
                color='yellow',
                width=3
            )
        ),
    })
    fig = go.Figure(data=dataFig, layout={
        'title': 'Rendement relatif ' + str(dfS[dfS['stockname'] == stockName]['name'].iloc[0]),
        'xaxis_title': "Heure",
        'yaxis_title': "%",
    })
    fig.update_layout(template="plotly_dark", xaxis_rangeslider_visible=False, showlegend=False)
    fig.show()

In [ ]:
for stockName in dfS[dfS['sellValue'].isna()]['stockname']: #dfS[dfS['stockname'] == 'CGG.PA']['stockname']: 
    computeRRgraph(stockName=stockName)

In [ ]:

import libsstock
importlib.reload(libsstock)
import updateDB
importlib.reload(updateDB)
from updateDB import updateOHLC, getStockIntradayData, getYFdata, updateDB
from libsstock import (
    loadStocks, computeIchimoku,
    graphIchimoku, graphBestGain, graphWorseGain, graphCashLock, graphRendement,
    graphEvolutionTitre, checkVar, graphDataForStock, loadStocks, graphGenericStock
)

#stockName = msg["text"].replace('/stockinfo','').replace('_', '.')
print('Generate data for ' + stockName)
if mybot is not None:
    mybot.sendMessage(
        chat_id, 'Generate data for ' + stockName
    )
graphDataA = []
#graphDataA.append(graphEvolutionTitre(stockName))
#graphDataA.append(graphDataForStock(stockName, freq=1, unit='W', histoDepth=timedelta(days=360)))
#graphDataA.append(graphDataForStock(stockName, freq=1, unit='D', histoDepth=timedelta(days=60)))
graphDataA.append(graphDataForStock(stockName, freq=5, unit='T', histoDepth=timedelta(days=0)))
if mybot is not None:
    for graphData in graphDataA:
        genIMGfromFile(graphData, 'img.png', scale=1.3, width=800, height=500)
        mybot.sendPhoto(chat_id, open('img.png', 'rb'))
else:
    for graphData in graphDataA:
        fig = go.Figure(graphData)
        fig.show()

In [ ]:
#display(ImageD(data=genIMGfromFile(graphBestGain(df), 'img.png', scale=1.3, width=500, height=500)))
#display(ImageD(data=genIMGfromFile(graphWorseGain(df), 'img.png', scale=1.3, width=500, height=500)))
#display(ImageD(data=genIMGfromFile(graphCashLock(df), 'img.png', scale=1.3, width=500, height=500)))
display(ImageD(data=genIMGfromFile(graphRendement(df, method='all'), 'img.png', scale=1.3, width=800, height=500)))

In [ ]:
for ind in range(len(df)):
    display(ImageD(data=genIMGfromFile(graphEvolutionTitre(getStockData(df.iloc[ind]['stockname']), df.iloc[ind]), 'img.png', scale=1.3, width=800, height=500)))

In [ ]:
# fill name in empty fields
#jsonToUpdate = 'stockprospects.json'
jsonToUpdate = 'mystocks.json'
df = pd.read_json(jsonToUpdate, orient='index')
#df= df.append({
#    'stockname': 'ALNOV.PA', 
#    'boughtDate': datetime(year=2020, month=2, day=17), 
#    'boughtValue': 2.07,
#    'boughtQ': 25,
#    'boughtFrais': 1.99,
#    'sellFrais': 1.99,
#}, ignore_index=True)
#df.loc[df['stockname'] == 'ALNOV.PA', 'name'] = 'NOVACYT'
#df.loc[df[df['stockname'] == 'ALNOV.PA'].iloc[-1].name, 'sellValue'] = 12.9
#df.loc[df[df['stockname'] == 'ALNOV.PA'].iloc[-1].name, 'sellDate'] = datetime(year=2020, month=2, day=20)
#df = fillNameFromYF(df)
df.to_json(jsonToUpdate, orient='index', indent=4)
display(df)
#historyData = getYFdate(df, startDate=date.today() - timedelta(days=365+41), stopDate=date.today() + timedelta(days=1))

In [ ]:
df.loc[((df['stockname'] == 'ALNOV.PA') && )]

In [ ]:
# fill name in empty fields
jsonToUpdate = 'stockprospects.json'
df = pd.read_json(jsonToUpdate, orient='index')
#df= df.append({'stockname': 'ALERS.PA'}, ignore_index=True)
#df.loc[df['stockname'] == 'ALERS.PA', 'name'] = 'EUROBIO SCIENT'
#df = fillNameFromYF(df)
df.to_json(jsonToUpdate, orient='index', indent=4)
display(df)
#historyData = getYFdate(df, startDate=date.today() - timedelta(days=365+41), stopDate=date.today() + timedelta(days=1))

In [ ]:
[{'label':df.loc[ind]['name'], 'value':df.loc[ind]['stockname']} for ind in df.index]

In [ ]:
dateStart = datetime.now() - timedelta(days=200)
dateStop = datetime.now()
df = pd.DataFrame([{'stockname': 'STM.PA',},])
dfData = loadStocks('mystocks.json')
dfData = dfData[dfData['stockname'] == df.iloc[0]['stockname']]
if len(dfData) > 0:
    df = dfData
else:
    df = fillNameFromYF(df)
dfHist = getYFdate(df, dateStart, dateStop)
dfHist = computeIchimoku(dfHist[0])
display(ImageD(data=genIMGfromFile(graphIchimoku(df, dfHist).update({'layout': dict(template="plotly_dark", xaxis_rangeslider_visible=False)}), 'img.png', scale=1.3, width=800, height=500)))

In [ ]:
listStockNames = [df.loc[ind]['stockname'] for ind in df.index]
for ind in range(len(listStockNames)):
    histo = computeIchimoku(historyData[ind])
    display(ImageD(data=genIMGfromFile(graphIchimoku(df[df['stockname'] == listStockNames[ind]], histo), 'img.png', scale=1.3)))